## Unscented Kalman Filter
In this project, a demonstration of the Unscented Kalman filter is presented to estimate the position of a vehicle with LIDAR & RADAR sensors. This demonstration is based on a simulator developed by Udacity and was completed as part of Udacity's Self-Driving Car Nanodegree. The [Udacity github repo](https://github.com/udacity/CarND-Extended-Kalman-Filter-Project) for this project has all the details about the software used in the simulator and the installation instructions.

### Summary of Setup Instructions
1. The project uses [uWebSocketIO](https://github.com/uNetworking/uWebSockets) for communication between the user-written algorithm and the simulator. Udacity has provided bash scripts to install this in both [Linux](https://github.com/chandrusuresh/CarND-Extended-Kalman-Filter-Project-master/blob/master/install-ubuntu.sh)/[Mac](https://github.com/chandrusuresh/CarND-Extended-Kalman-Filter-Project-master/blob/master/install-mac.sh) environments. These scripts are included in this repo.
2. The simulator can be downloaded from [here](https://github.com/udacity/self-driving-car-sim/releases).

### Basic Build Instructions (Linux/Mac)
1. Clone this repo.
2. Make a build directory: `mkdir build && cd build`
3. Compile: `cmake .. && make`
4. Run it: `./UnscentedKF`
This should setup a listener for the code to get data from the simulator.
5. Launch the simulator from a terminal: `./`
6. Select the Kalman Filter project and click start to start the simulation.

These steps should get any user in a Linux/Mac environment up and running with the code.

## Constant Turn Rate & Velocity Model
In this project, the plant (car) dynamics is modeled by the constant turn rate & velocity (CTRV) model. The state vector at time sample $k$ is represented as $x_k$ with 5 components: positions in the $x$ & $y$ coordinates $p_x$, $p_y$ and translational velocity $v$, the vehicle yaw $\psi$ and its rate $\dot{\psi}$.
The state transition model (plant dynamics) is defined by the following discrete time state space form.
$$ \begin{align*} \left[ \begin{matrix}  p_x\\ p_y \\ v \\\psi \\ \dot{\psi} \end{matrix} \right]_{t=k+1} = \left[ \begin{matrix} v \cos \psi \Delta t \\ v \sin \psi \Delta t \\ 0 \\ \dot{\psi} \Delta t \\ 0\end{matrix}\right]_{t=k} + \left[ \begin{matrix}  \frac{\Delta t^2 \cos \psi a }{2}\\ \frac{\Delta t^2 \sin \psi a }{2} \\ \Delta t a \\ \frac{\Delta t^2 a_{\psi}}{2} \\ \Delta t a_{\psi} \end{matrix} \right] \\ \end{align*}$$
where $[\begin{matrix} a & a_{\psi} \end{matrix}]^T$ is the $\textit{i.i.d}$  zero mean process noise from accelerations. $\Delta t$ is the sampling time elapsed between time samples $k$ and $k+1$. Note that while the plant is modeled by the CTRV model, the accelerations are modeled as noise. 

Note that the function mapping acceleration noise to the state variables is also nonlinear. The state variable is therefore augmented with dummy variables to include this nonlinearith in the state transition function.
$$ \begin{align*} \left[ \begin{matrix}  p_x\\ p_y \\ v \\\psi \\ \dot{\psi} \\ a \\ a_{\psi}\end{matrix} \right]_{t=k+1} = \left[ \begin{matrix} v \cos \psi \Delta t + \frac{\Delta t^2 \cos \psi a }{2} \\ v \sin \psi \Delta t + \frac{\Delta t^2 \sin \psi a }{2} \\ \Delta t a \\ \dot{\psi} \Delta t + \frac{\Delta t^2 a_{\psi}}{2}\\ \Delta t a_{\psi}\\ 0\\ 0\end{matrix}\right]_{t=k} \end{align*}$$
The above plant model is expressed concisely as:
$$ \begin{equation} x_{k+1} = F(x_k) \tag{1} \end{equation}$$
where $F(\cdot)$ is the nonlinear function described above.

## Sensor Model
In this project, a LIDAR and a RADAR sensors are modeled at different sample rates. The measurement model for these two sensors is described below.

### LIDAR
LIDAR (stands for LIght Detection And Ranging) is a time of flight sensor used for measuring distances between nearby objects by illuminating them with laser light and measuring the time taken for the reflected signal to be received by a sensor. A LIDAR sensor therefore measures the distance to a given object in 3D space. In this project, we work with 2D data and are only interested in the $x$ & $y$ coordinate frames.

The LIDAR sensor measurement model is defined as follows: The measurement vector $z_l$ is the vector of distances in the $x$ & $y$ coordinate frames represented by $z_{lx}$ & $z_{ly}$.
$$ \left[ \begin{matrix}  z_{lx} \\ z_{ly} \end{matrix} \right]_{t=k} = \left[ \begin{matrix} 1&0&0&0&0&0&0 \\ 0&1&0&0&0&0&0 \end{matrix} \right] \left[ \begin{matrix}  p_x\\ p_y \\ v \\\psi \\ \dot{\psi} \\ a \\ a_{\psi} \end{matrix} \right]_{t=k} + v_{l,k} $$
where $v_l \sim \mathcal{N}(0,R_l) $ is $\textit{i.i.d}$ measurement noise. $R_l$ is the measurement covariance and in this project $R_l = \left[ \begin{matrix} 0.0225 & 0 \\ 0 & 0.0225\end{matrix} \right]$.
The concise expression for the LIDAR measurement model is: $$\begin{equation} z_{l,k} = H_l x_k + v_{l,k} \tag{2} \end{equation}$$

### RADAR
RADAR (stands for RAdio Detection And Ranging) is an object detection sensor that transmits radio waves in all directions and receives and processes the reflected waves to determine to determine the location of nearby objects. A RADAR sensor measures 3 quantities - the range $r$ of an object from the sensor, the bearing $\psi$ of the object from the sensor and the relative velocity $\dot{r}$ at which the object is moving w.r.t the sensor.

The RADAR sensor measurement model is defined as follows: The measurement vector $z_{r} = \left[ \begin{matrix} r & \psi & \dot{r} \end{matrix} \right]^T$.

$$ \left[ \begin{matrix}  r \\ \theta \\ \dot{r} \end{matrix} \right]_{t=k} = \left[ \begin{matrix} \sqrt{p_x^2 + p_y^2} \\ \tan^{-1}\left(\frac{p_y}{p_x}\right) \\ \frac{p_x v \cos \theta + p_y v \sin \theta}{\sqrt{p_x^2 + p_y^2}}\end{matrix} \right]_{t=k} + v_{r,k}$$
where $v_r \sim \mathcal{N}(0,R_r) $ is $\textit{i.i.d}$ measurement noise. $R_r$ is the measurement covariance and in this project $R_r = \left[ \begin{matrix} 0.09 & 0 & 0 \\ 0 & 0.0009 & 0 \\ 0 & 0 & 0.09 \end{matrix} \right]$.
The concise expression for the RADAR measurement model is: $$\begin{equation} z_{r,k} = H_r(x_k) + v_{r,k} \tag{3} \end{equation}$$ where $H_r(\cdot)$ is the nonlinear matrix expression above for the RADAR sensor model.



## Unscented Kalman Filter (UKF)
The Unscented Kalman filter belongs to a class of filters based on Linear Regression which use a statistical linearization technique. The linearization is performed by linearizing a nonlinear function by linear regression of $n$ points drawn apriori from a distribution of a random variable. The UKF uses a deterministic sampling technique to pick a minimal set of "sigma" points around the current posterior mean. The sigma points are then propagated through the nonlinear function from which a new mean and covariance are computed. The resulting model more accurately estimates the mean and covariance. Another advantage of this technique over the EKF is that it does not require Jacobian computations.

In this project, we use the LIDAR and RADAR as sensors to update the position and velocity of a vehicle.


### Generate Sigma Points
The optimum number of sigma points for linear regression is $2n+1$ where $n$ is the number of state variables. In this project, we have 5 active and 2 dummy variables, so $n_\chi = 2(5+2)+1 = 15$.

These $15$ sigma points are computed as follows:
1. $\chi_{0,k+1} = x_{k|k}$ 
2. $\chi_{1:7,k+1} = x_{k|k} + \sqrt{(\lambda+n_x)P_{k|k}}$
3. $\chi_{8:14,k+1} = x_{k|k} - \sqrt{(\lambda+n_x)P_{k|k}}$

where $x_{k|k}$ & $P_{k|k}$ are the posterior mean and covariance of the current estimate, $n_x = 7$ is the number of state variables and $\lambda = 3-n_x$. Each $\chi$ in steps 2-3 above is a column from the right-hand side of the corresponding expression.

The square root of the posterior covariance is computed by Cholesky factorization.

### Sigma Point Prediction 
The above sigma points are propagated through the non-linear state transition matrix to compute the "next states". The mean and covariance of the prediction is given as a weighted average over all the sigma points.

$$ \chi_{i,k+1|k} = F(\chi_{k|k})$$
$$ x_{k+1|k} = \sum_{i=0}^{n_{\chi}} w_i \chi_{i,k+1|k}$$
where $w_i$ is the weight of the $i^{th}$ sigma point. These weights are constant with time and can be pre-computed as:
1. $w_{0} = \frac{\lambda}{\lambda+n_x}$ 
2. $w_{1:14} = \frac{0.5}{\lambda+n_x}$ 

The predicted covariance at the next time step is computed as:
$$ P_{k+1|k} = \sum_{i=0}^{n_{\chi}} w_i (\chi_i - x_{k+1|k}) (\chi_i - x_{k+1|k})^T$$

This completes the prediction step.

### Measurement Update
The measurement is predicted for each sigma point and the weighted mean is computed as follows:
1. $\hat{z}_{i,k+1|k} = H(\chi_{i,k+1|k})$
2. $\hat{z}_{k+1|k} = \sum_{i=0}^{n_{\chi}} w_i \hat{z}_{i,k+1|k}$

The predicted measurement covariance is then computed:
$$\delta_{i,\hat{z}} = \hat{z}_{i,k+1|k} - \hat{z}_{k+1,k}$$
$$S =  \sum_{i=0}^{n_{\chi}} w_i \delta_{i,\hat{z}} \delta_{i,\hat{z}}^T$$

The cross-correlation matrix between the predicted sigma points and measurement is computed as:
$$\delta_{i,x} = \chi_{i,k+1|k} - x_{k+1,k}$$
$$ T_c = \sum_{i=0}^{n_{\chi}} w_i \delta_{i,x} \delta_{i,\hat{z}}^T$$

The Kalman gain is computed as $K_{k+1} = T_c S^{-1}$

The posterior estimate of the state is given by: $x_{k+1|k+1} = x_{k|k} + K_{k+1} (z-\hat{z}_{k+1|k})$

The posterior estimate of the process covariance is given by: $P_{k+1|k+1} = P_{k+1|k} - K_{k+1} S K_{k+1}^T$
